In [ ]:
def load_images_and_labels(root_dir):
    """
    Load images and labels from the specified directory.
    Each subdirectory corresponds to a class label.
    """
    images = []
    labels = []
    class_names = []  # To store unique class labels

    for subdir in ["Faces/Faces", "Original Images/Original Images"]:
        folder_path = os.path.join(root_dir, subdir)
        for class_name in os.listdir(folder_path):
            class_dir = os.path.join(folder_path, class_name)
            if os.path.isdir(class_dir):
                class_names.append(class_name)  # Add class name
                for file_name in os.listdir(class_dir):
                    file_path = os.path.join(class_dir, file_name)
                    image = preprocess_image(file_path)
                    if image is not None:  # Only add valid images
                        images.append(image)
                        labels.append(class_name)

    return np.array(images), labels, class_names

def preprocess_image(image_path):
    """
    Preprocess the input image: resize, convert to grayscale, and normalize.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not load image at {image_path}")
        return None
    resized_image = cv2.resize(image, (128, 128))  # Resize to 128x128 pixels
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    normalized_image = grayscale_image / 255.0  # Normalize pixel values to [0, 1]
    return normalized_image

# Dataset path
dataset_path = r"C:\Users\abhij\Downloads\Images\Splitted Images\test"
data_images, labels, class_names = load_images_and_labels(dataset_path)

# Encode labels to numerical format
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(labels)

In [56]:
def preprocess_image(image_path):
    """Preprocess the input image: resize, convert to grayscale, and normalize."""
    image = cv2.imread(image_path)  # Read the image
    if image is None:
        print(f"Error: Could not load image at {image_path}.")
        return None
    resized_image = cv2.resize(image, (128, 128))  # Resize to 128x128 pixels
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    normalized_image = grayscale_image / 255.0  # Normalize pixel values to [0, 1]
    return normalized_image

In [59]:
# Step 3: Feature Extraction
# -----------------------------------------------
def extract_features(image):
    """Extract all features from the preprocessed image."""
    # Chain code, GLCM, Regional, and LBP features
    contours, _ = cv2.findContours((image * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    chain_features = np.zeros(128)
    if contours:
        chain_features[:len(contours[0].ravel())] = contours[0].ravel()[:128]

    glcm = graycomatrix((image * 255).astype(np.uint8), distances=[1], angles=[0], levels=256)
    glcm_features = [
        graycoprops(glcm, 'contrast')[0, 0],
        graycoprops(glcm, 'energy')[0, 0],
        graycoprops(glcm, 'homogeneity')[0, 0]
    ]

    area = sum(cv2.contourArea(c) for c in contours)
    perimeter = sum(cv2.arcLength(c, True) for c in contours)
    regional_features = [area, perimeter]

    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    lbp_features = lbp_hist / np.sum(lbp_hist)

    return np.concatenate((chain_features, glcm_features, regional_features, lbp_features))





In [60]:
# Extract features for all images
features = np.array([extract_features(img) for img in data_images])

/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [61]:
# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [66]:
# Step 4: ANN for Classification
# -----------------------------------------------
# Convert labels to categorical format
categorical_labels = to_categorical(numerical_labels, num_classes=len(class_names))

model = Sequential([
    Dense(512, input_dim=scaled_features.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the ANN model
model.fit(scaled_features, categorical_labels, epochs=200, batch_size=8, validation_split=0.2)

Epoch 1/200


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0668 - loss: 3.5264 - val_accuracy: 0.0019 - val_loss: 6.7820
Epoch 2/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1043 - loss: 3.1645 - val_accuracy: 0.0000e+00 - val_loss: 7.5626
Epoch 3/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1066 - loss: 3.0815 - val_accuracy: 0.0000e+00 - val_loss: 8.1154
Epoch 4/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1085 - loss: 3.0452 - val_accuracy: 0.0000e+00 - val_loss: 8.5619
Epoch 5/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1434 - loss: 2.9557 - val_accuracy: 0.0039 - val_loss: 9.0025
Epoch 6/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1557 - loss: 2.9639 - val_accuracy: 0.0039 - val_loss: 9.3744
Epoch 7/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1500 - loss: 2.9179 - val_accuracy: 0.0000e+00 - val_loss: 9.2150
Epoch 8/200
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1462 - loss: 2.8973 - val_

In [ ]:
def test_model(new_image_path):
    """Test the model with a new image."""
    new_image = preprocess_image(new_image_path)
    new_features = extract_features(new_image)
    scaled_new_features = scaler.transform([new_features])

    prediction = model.predict(scaled_new_features)
    predicted_label = class_names[np.argmax(prediction)]
    print(f"The face is recognized as: {predicted_label}")

# Replace with your test image path
test_image_path = r"C:\Users\abhij\Downloads\Images\Splitted Images\test\Original Images\Robert Downey Jr\Robert Downey Jr_91.jpg"
test_model(test_image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
The face is recognized as: Virat Kohli


/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [ ]:
# Save the model to a .pkl file
with open('face_recognition_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as 'face_recognition_model.pkl'")